In [1]:
import os
import sys
import tarfile
import tensorflow as tf
import numpy as np
from scipy import ndimage
from six.moves import cPickle as pickle
from six.moves.urllib.request import urlretrieve
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
top_data_folder = './data/notMNIST/'
dataset_small = 'notMNIST_small'
dataset_large = 'notMNIST_large'

logs_folder = './logs'

if not os.path.exists(top_data_folder):
    os.makedirs(top_data_folder)
    
def path_to(f):
    return os.path.join(top_data_folder, f)
    
if not os.path.exists(logs_folder):
    os.makedirs(logs_folder)

In [3]:
class DownloadProgress:
    def __init__(self):
        self.last_percent_reported = None

    def __call__(self, count, blockSize, totalSize):
        percent = int(count * blockSize * 100 / totalSize)

        if self.last_percent_reported != percent:
            if percent % 5 == 0:
                sys.stdout.write("%s%%" % percent)
                sys.stdout.flush()
            else:
                sys.stdout.write(".")
                sys.stdout.flush()
      
            self.last_percent_reported = percent

In [4]:
def download_and_extract(dataset_name):
    dataset_archive = dataset_name + '.tar.gz'
    filename = path_to(dataset_archive)
    if not os.path.exists(filename):
        url = 'http://commondatastorage.googleapis.com/books1000/' + dataset_archive
        urlretrieve(url, filename, reporthook=DownloadProgress())

    letters_folder = path_to(dataset_name)

    if not os.path.exists(letters_folder):
        with tarfile.open(filename) as tar:
            tar.extractall(path=top_data_folder)

In [5]:
download_and_extract(dataset_small)

In [6]:
download_and_extract(dataset_large)

In [7]:
image_size = 28
image_pixels = image_size * image_size
class_count = 10

def load_folder(folder):
    image_filenames = [f for f in os.listdir(folder) if not os.path.isdir(os.path.join(folder, f))]
    image_count = len(image_filenames)
    
    dataset = np.ndarray(shape=(image_count, image_pixels), dtype=np.float32)
    
    image_counter = 0
    
    for f in image_filenames:
        image_filename = os.path.join(folder, f)
        try:
            image_data = np.ndarray.flatten(ndimage.imread(image_filename).astype(float))
            dataset[image_counter, :] = image_data
            image_counter += 1
        except IOError as e:
            # There are plenty of images, I skip this one
            pass
        
    dataset = dataset[0:image_counter, :]
    
    print('Full dataset tensor for %s:' % folder, dataset.shape)
    
    return dataset

In [8]:
def get_complete_dataset(dataset_name):
    image_depth = 256
    
    pickle_file = path_to(dataset_name + '.pickle')
    
    if not os.path.exists(pickle_file):
        image_pixels = []
        image_labels = []
        image_labels_one_hot = []
        
        for i, letter in enumerate("ABCDEFGHIJ"):
            letter_dataset = (load_folder(path_to(os.path.join(dataset_name, letter))) * 2 - image_depth) / image_depth
            
            letter_image_count = len(letter_dataset)
            
            letter_labels = np.ones(shape=(letter_image_count,), dtype=np.float32) * i
            letter_labels_one_hot = np.ndarray(shape=(letter_image_count, class_count), dtype=np.float32)
            
            label_one_hot = np.zeros(shape = (class_count,), dtype=np.float32)
            label_one_hot[i] = 1.0
            letter_labels_one_hot[:] = label_one_hot
            
            image_pixels.append(letter_dataset)
            image_labels.append(letter_labels)
            image_labels_one_hot.append(letter_labels_one_hot)
            
        dataset = { 'pixels': np.concatenate(image_pixels), 
                    'labels': np.concatenate(image_labels),
                    'labels_one_hot': np.concatenate(image_labels_one_hot) }
            
        with open(pickle_file, 'wb') as f:
            pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
            
        return dataset
    else:
        with open(pickle_file, 'rb') as f:
            return pickle.load(f)

In [9]:
def split(dataset_name, test_size=0.2, validate_size=0.25, random_state=42):
    dataset = get_complete_dataset(dataset_name)
    answer = {}
    
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_validate_indices, test_indices = list(sss1.split(dataset['pixels'], dataset['labels']))[0]

    answer['X_test'] = dataset['pixels'][test_indices]
    answer['y_test'] = dataset['labels_one_hot'][test_indices]

    train_validate_pixels = dataset['pixels'][train_validate_indices]
    train_validate_labels = dataset['labels'][train_validate_indices]
    train_validate_labels_one_hot = dataset['labels_one_hot'][train_validate_indices]
    
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=validate_size, random_state=random_state * random_state)
    train_indices, validate_indices = list(sss2.split(train_validate_pixels, train_validate_labels_one_hot))[0]
    
    answer['X_train'] = train_validate_pixels[train_indices]
    answer['y_train'] = train_validate_labels_one_hot[train_indices]
    
    answer['X_validate'] = train_validate_pixels[validate_indices]
    answer['y_validate'] = train_validate_labels_one_hot[validate_indices]
    
    return answer    

In [10]:
split_dataset = split(dataset_small)
#split_dataset = split(dataset_large)

In [11]:
for k in split_dataset.keys():
    print(k, split_dataset[k].shape)

('X_test', (3745, 784))
('X_train', (11234, 784))
('X_validate', (3745, 784))
('y_validate', (3745, 10))
('y_train', (11234, 10))
('y_test', (3745, 10))


In [12]:
batch_size = 128
hidden_node_count = 1024
learning_rate = 0.01

def accuracy(predictions, labels):
    correct_predictions = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
    return 100.0 * tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_pixels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, class_count))
    tf_validate_dataset = tf.constant(split_dataset['X_validate'])
    tf_validate_labels = tf.constant(split_dataset['y_validate'])
    tf_test_dataset = tf.constant(split_dataset['X_test'])
    tf_test_labels = tf.constant(split_dataset['y_test'])
    
    weights_0 = tf.Variable(tf.truncated_normal([image_pixels, hidden_node_count]))
    biases_0 = tf.Variable(tf.zeros([hidden_node_count]))
    weights_1 = tf.Variable(tf.truncated_normal([hidden_node_count, class_count]))
    biases_1 =tf.Variable(tf.zeros([class_count]))

    hidden_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights_0) + biases_0)
    logits_train = tf.matmul(hidden_train, weights_1) + biases_1
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_train, tf_train_labels))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
    prediction_train = tf.nn.softmax(logits_train)
    
    hidden_validate = tf.nn.relu(tf.matmul(tf_validate_dataset, weights_0) + biases_0)
    prediction_validate = tf.nn.softmax(tf.matmul(hidden_validate, weights_1) + biases_1)
    
    hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_0) + biases_0)
    prediction_test = tf.nn.softmax(tf.matmul(hidden_test, weights_1) + biases_1)
    
    accuracy_train = accuracy(prediction_train, tf_train_labels)
    accuracy_validate = accuracy(prediction_validate, tf_validate_labels)
    accuracy_test = accuracy(prediction_test, tf_test_labels)
    
    tf.scalar_summary("Minibatch Loss", loss)
    tf.scalar_summary("Minibatch Accuracy", accuracy_validate)
    tf.scalar_summary("Validate Accuracy", accuracy_validate)
    tf.scalar_summary("Test Accuracy", accuracy_test)
    
    summary_op = tf.merge_all_summaries()

In [13]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    writer = tf.train.SummaryWriter(logs_folder, graph=graph)
    
    session.run(tf.initialize_all_variables())
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (split_dataset['y_train'].shape[0] - batch_size)
    
        batch_data = split_dataset['X_train'][offset:(offset + batch_size), :]
        batch_labels = split_dataset['y_train'][offset:(offset + batch_size), :]
    
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, summary, acc_tr, acc_val, acc_te = session.run(
            [optimizer, loss, prediction_train, summary_op, 
             accuracy_train, accuracy_validate, accuracy_test], 
            feed_dict=feed_dict)
        
        writer.add_summary(summary, step)
        
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % acc_tr)
            print("Validation accuracy: %.1f%%" % acc_val)
    print("Test accuracy: %.1f%%" % acc_te)

Initialized
Minibatch loss at step 0: 640.280334
Minibatch accuracy: 12.5%
Validation accuracy: 12.7%
Minibatch loss at step 1000: 12.037876
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 2000: 0.807581
Minibatch accuracy: 96.1%
Validation accuracy: 91.3%
Minibatch loss at step 3000: 0.000766
Minibatch accuracy: 100.0%
Validation accuracy: 92.5%
Minibatch loss at step 4000: 0.000162
Minibatch accuracy: 100.0%
Validation accuracy: 93.0%
Minibatch loss at step 5000: 0.000005
Minibatch accuracy: 100.0%
Validation accuracy: 93.1%
Test accuracy: 84.6%
